# End Pointing of Speech signal

* Based on [Paper by Rabiner and Sambur](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=6778857)

In [ ]:
%matplotlib inline
from scipy import signal
from scipy.io.wavfile import write,read
import matplotlib.pyplot as plt
import math
import numpy as np
import IPython
from numpy import convolve
from scipy.signal import hamming
from scipy.ndimage.interpolation import shift
from scipy.linalg import toeplitz,inv
from scipy.fftpack import fft,fftshift,ifft

# Returns ndarray corresponding to voiced part in filename wav file
def endpointer(filename,Fs,glob_count=1):
    Ts=1/Fs
    read_wav = read(filename+'.wav')
    inp=np.array(read_wav[1],dtype='float64')
    plt.rcParams["figure.figsize"] = (18,10)
    #plt.title("Raw Input")
    #plt.plot(inp,'blue')
    
    #------------------------------------------------------------------------------
    #Compute Avg magnitude (Short Time), with window size N/20 (0-9 spoken twice)
    W=40
    
    pad_zeros=np.zeros(W)
    zp_inp_t=np.append(pad_zeros,inp)
    zp_inp=np.append(zp_inp_t,pad_zeros)
    st_avg=[np.sum(abs(zp_inp[i-W:i+W])) for i in range(W,zp_inp.size-W)]
    #plt.plot(st_avg,'red')

    #Compute ZCR
    st_zcr=[np.where(np.diff(np.signbit(zp_inp[i-W:i+W])))[0].size for i in range(W,zp_inp.size-W)]
    #plt.plot(st_zcr,'green')
    
    #Compute IZCT assuming 100ms silence in start
    num_samp_in_sil=int(Fs*0.01)
    sil_inp=inp[::-1][0:num_samp_in_sil]
    sil_inp_t=np.append(pad_zeros,sil_inp)
    sil_inp_wpad=np.append(sil_inp_t,pad_zeros)
    #Compute ZCR
    sil_zcr=[np.where(np.diff(np.signbit(sil_inp_wpad[i-W:i+W])))[0].size for i in range(W,sil_inp_wpad.size-W)]
    sil_avg=[np.sum(np.abs(sil_inp_wpad[i-W:i+W])) for i in range(W,sil_inp_wpad.size-W)]
    
    
    sigma_izc=np.std(sil_zcr)
    mean_izc=np.amax(sil_zcr)
    IF=25
    
    IZCT=min(IF,mean_izc+(2*sigma_izc))
    #print (mean_izc+(2*sigma_izc))
    
    #Compute Peak Energy Imax and Silence Energy Imin
    Imx=np.amax(st_avg)
    Imn=np.mean(sil_avg)
    #print(Imn)
    #Compute ITL and ITU
    I1=0.001*(Imx-Imn)+Imn
    I2=1*Imn
    #print(I1)
    #print(I2)
    #ITL=min(I1,I2)
    ITL=12250
    print(ITL)
    ITU=5*ITL
    
    #Search Fwd
    i=0
    while(True):
        #print(i)
        if(i==0):
            start_index=num_samp_in_sil+1
        zero_st,zero_fi=search(st_avg,st_zcr,start_index,IZCT,ITU,ITL)
        #print (zero_fi)
        if(zero_st==-1):
            start_index=zero_fi
            continue
        filename="./Processed_Data/"+str(int(i/2))+"/"+str(glob_count+(i%2))+".wav"
        print (filename+" Indices "+str(zero_st)+":"+str((zero_fi)))
        end_point_sig=inp[zero_st:zero_fi]
        
        pemph_sig = np.zeros(zero_fi-zero_st);
        a=0.97
        for k in range (zero_fi-zero_st):
            if(k==0):
                pemph_sig[k]=end_point_sig[k]
            else:
                pemph_sig[k]=end_point_sig[k] - a*end_point_sig[k-1];

        
        write(filename, 8000, np.array(pemph_sig).astype(np.dtype('i2')))
        start_index=zero_fi
        i=i+1
        if(i==20):
            break
        #plt.plot(inp[zero_st:zero_fi])
        
endpointer('./Raw_Data/Digits_male_8Khz/zero_to_nine_Jatin1',8000,13)
endpointer('./Raw_Data/Digits_male_8Khz/zero_to_nine_Jatin2',8000,15)


12250


In [60]:
def search(st_avg,st_zcr,start_index,IZCT,ITU,ITL):
    N1=0
    N2=0
    m=start_index    
    #Fwd Search
    while(True):
        flag=0
        contender=0
        i=0
        #print(st_avg[m]-ITL)
        if(st_avg[m] >= ITL):
            i=m
            while(True):
                if(st_avg[i]<ITL):
                    flag=1
                    break
                if(st_avg[i]>=ITU):
                    contender=i
                    if(i==m):
                        contender=contender-1
                    flag=2
                    break
                else: 
                    i=i+1
            
            if(flag==2):              
                N1=contender
                break
        if(flag==1):
            m=i+1
            continue
        if(flag==0):
            m=m+1
            continue

                
    #Bkwd Search
    m=N1
    while(True):
        flag=0
        contender=0
        i=0
        if(st_avg[m] <= ITU):
            i=m
            while(True):
                if(st_avg[i]>ITU):
                    flag=1
                    break
                if(st_avg[i]<=ITL):
                    contender=i
                    if(i==m):
                        contender=contender+1
                    flag=2
                    break
                else: 
                    i=i+1
            
            if(flag==2):
                N2=contender
                break
                
        if(flag==1):
            m=i+1
        if(flag==0):
            m=m+1
    
    if((N2-N1)<1600):
        return -1,N2+1
    return N1,N2
            
    
 

## Combinations of ITL and N2-N1 diff used

       
| Person |   ITL    |  N2-N1  |      
|--------|----------|---------|  
| Hitesh |  2250    |   1600  | 
|'i'     |  70.84   |  -2.85  | 
|'n'     |  63.73   |   1.07  |
|'s'     |  70.02   |  -0.25  |        
